We will collect text data using Twitter API.

In [2]:
!pip install tweepy

In [3]:
!pip install openpyxl

In [4]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [5]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [6]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [7]:
query = "RCB winning IPL"

In [8]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x000001EA30D842B0>, _json={'created_at': 'Sat Sep 25 08:00:43 +0000 2021', 'id': 1441674008349462528, 'id_str': '1441674008349462528', 'full_text': 'IPL TV coverage is a series of commercials with some cricket in between.\nWhile the game is progressing along similar lines (MI and CSK winning, RCB losing), good to see new ads every year', 'truncated': False, 'display_text_range': [0, 187], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 52306451, 'id_str': '52306451', 'name': 'D Prasanth Nair', 'screen_name': 'DPrasanthNair', 'location': 'Mumbai', 'description': 'Doctorate-IIMA,Angel I

Get the tweets and some Attributes

In [9]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [10]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [11]:
stream(data=["RCB winning IPL"] , file_name='my_tweets')

In [12]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,IPL TV coverage is a series of commercials wit...,D Prasanth Nair,57995,8977,Mumbai,False,1,0,2021-09-25 08:00:43
1,Reason behind RCB not winning IPL trophy yet h...,ladki_baaz,261,37,,False,2,0,2021-09-25 07:30:51
2,“Got to get on that winning run again. This on...,Circle of Cricket,88138,142164,,True,11,1,2021-09-25 07:25:10
3,IPL Teams winning %\nCSK is in top with 60.48 ...,Sportsmatik,4806,15103,India,False,2,0,2021-09-25 06:18:25
4,RT @CricBloggers: #RCB has been a trailtail.\n...,#StayHome,908109,7696,കേരളം,False,0,1,2021-09-25 04:09:02


In [13]:
!pip install textblob

Sentiment(polarity, subjectivity)

Polarity score: [-1 , +1]

Subjectivity: [0.0 , 1.0] 
where 0.0 is very Objective and 1.0 is very Subjective.

In [15]:
from textblob import TextBlob

In [16]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [17]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [18]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [19]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,IPL TV coverage is a series of commercials wit...,D Prasanth Nair,57995,8977,Mumbai,False,1,0,2021-09-25 08:00:43,IPL TV coverage is a series of commercials wit...
1,Reason behind RCB not winning IPL trophy yet h...,ladki_baaz,261,37,,False,2,0,2021-09-25 07:30:51,Reason behind RCB not winning IPL trophy yet
2,“Got to get on that winning run again. This on...,Circle of Cricket,88138,142164,,True,11,1,2021-09-25 07:25:10,Got to get on that winning run again This one ...
3,IPL Teams winning %\nCSK is in top with 60.48 ...,Sportsmatik,4806,15103,India,False,2,0,2021-09-25 06:18:25,IPL Teams winning CSK is in top with 60 48 Per...
4,RT @CricBloggers: #RCB has been a trailtail.\n...,#StayHome,908109,7696,കേരളം,False,0,1,2021-09-25 04:09:02,RT RCB has been a trailtail Similar to IPL2020...


In [20]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,IPL TV coverage is a series of commercials wit...,D Prasanth Nair,57995,8977,Mumbai,False,1,0,2021-09-25 08:00:43,IPL TV coverage is a series of commercials wit...,Negative
1,Reason behind RCB not winning IPL trophy yet h...,ladki_baaz,261,37,,False,2,0,2021-09-25 07:30:51,Reason behind RCB not winning IPL trophy yet,Negative
2,“Got to get on that winning run again. This on...,Circle of Cricket,88138,142164,,True,11,1,2021-09-25 07:25:10,Got to get on that winning run again This one ...,Positive
3,IPL Teams winning %\nCSK is in top with 60.48 ...,Sportsmatik,4806,15103,India,False,2,0,2021-09-25 06:18:25,IPL Teams winning CSK is in top with 60 48 Per...,Positive
4,RT @CricBloggers: #RCB has been a trailtail.\n...,#StayHome,908109,7696,കേരളം,False,0,1,2021-09-25 04:09:02,RT RCB has been a trailtail Similar to IPL2020...,Positive


In [21]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Reason behind RCB not winning IPL trophy yet https://t.co/B0hA8l6iIQ

Clean tweet:
 Reason behind RCB not winning IPL trophy yet

Sentiment of the tweet:
 Negative


In [22]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @Bounce_Bully Me as well get a lot of hate for DDP comments. 

One section of Kohli &amp; RCB fans are more invested in… https://t.co/3NgZtILLVm

Clean tweet:
 Bully Me as well get a lot of hate for DDP comments One section of Kohli amp RCB fans are more invested in

Sentiment of the tweet:
 Negative


In [23]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @CricCrazyJohns @LoyalSachinFan Why u blocked muffa? If u don't reply mi rcb winning ipl

Clean tweet:
 Why u blocked muffa If u don t reply mi rcb winning ipl

Sentiment of the tweet:
 Positive


In [24]:
df[df.Sentiment == 'Positive'].shape[0]

624

In [25]:
df[df.Sentiment == 'Neutral'].shape[0]

40

In [26]:
df[df.Sentiment == 'Negative'].shape[0]

59